<a href="https://colab.research.google.com/github/Gaeli75/Machine-Learning-Pratices/blob/main/Customer%20Churn%20Prediction%20Using%20Random%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# Load the cleaned dataset
df = pd.read_csv('/content/clean_data_after_eda.csv')

In [5]:
# Convert date columns to datetime format
date_columns = ["date_activ", "date_end", "date_modif_prod", "date_renewal"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
    print(df.dtypes)

id                                        object
channel_sales                             object
cons_12m                                   int64
cons_gas_12m                               int64
cons_last_month                            int64
date_activ                        datetime64[ns]
date_end                                  object
date_modif_prod                           object
date_renewal                              object
forecast_cons_12m                        float64
forecast_cons_year                         int64
forecast_discount_energy                 float64
forecast_meter_rent_12m                  float64
forecast_price_energy_off_peak           float64
forecast_price_energy_peak               float64
forecast_price_pow_off_peak              float64
has_gas                                   object
imp_cons                                 float64
margin_gross_pow_ele                     float64
margin_net_pow_ele                       float64
nb_prod_act         

In [6]:
# FEATURE ENGINEERING: Creating new features from dates
# -----------------------------------------------------------------------------

# Extracting components from 'date_activ' to capture trends related to time
df["year_activ"] = df["date_activ"].dt.year      # Useful for identifying yearly trends
df["month_activ"] = df["date_activ"].dt.month    # Some months may have seasonal effects
df["day_activ"] = df["date_activ"].dt.day        # Not as important but could be useful


In [7]:
# Calculate how long the customer has been active
df["days_since_activation"] = (pd.to_datetime("today") - df["date_activ"]).dt.days
# Why? Helps understand the longevity of a customer, which can be linked to churn

In [8]:
# Extract similar features for renewal date
df["year_renewal"] = df["date_renewal"].dt.year
df["month_renewal"] = df["date_renewal"].dt.month

In [9]:
# Number of days between the last product modification and renewal
df["time_since_last_modif"] = (df["date_renewal"] - df["date_modif_prod"]).dt.days
# Why? Frequent modifications might indicate dissatisfaction, impacting churn.

In [10]:
# Contract duration (how long a customer was subscribed)
df["contract_duration"] = (df["date_end"] - df["date_activ"]).dt.days
# Why? Shorter contracts might indicate higher churn risk.

In [12]:
# LOADING PRICE DATA AND CREATING PRICE DIFFERENCE FEATURES
# -----------------------------------------------------------------------------

# Load the dataset containing price information
price_df = pd.read_csv('/content/price_data .csv')

In [13]:
# Convert 'price_date' to datetime
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')

In [14]:
# Aggregate price data by 'id' and 'price_date' to get monthly mean values
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({
    'price_off_peak_var': 'mean',  # Variable price for off-peak energy
    'price_off_peak_fix': 'mean'   # Fixed price for off-peak power
}).reset_index()

In [15]:
# Extract January and December prices for each 'id'
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()  # First entry per ID = Jan price
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()   # Last entry per ID = Dec price

In [16]:
# Calculate the difference in off-peak prices between December and January
diff = pd.merge(
    dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}),
    jan_prices.drop(columns='price_date'),
    on='id'
)

In [17]:
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']

In [18]:
# Keep only relevant columns
diff = diff[['id', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']]

In [19]:
# Merge the computed price difference back into the main dataframe
df = df.merge(diff, on='id', how='left')

# INTERACTION FEATURES: Combining Categorical Variables


In [20]:
# Create a combined feature if these columns exist
if "contract_type" in df.columns and "customer_type" in df.columns:
    df["contract_type_customer_interaction"] = df["contract_type"].astype(str) + "_" + df["customer_type"].astype(str)
# Why? Certain contract types may be more popular among specific customer groups, influencing churn.


In [21]:
# Remove original date columns since we have extracted useful features
drop_cols = ["date_activ", "date_end", "date_modif_prod", "date_renewal"]
df.drop(columns=drop_cols, inplace=True)

In [28]:
dft3 = df.copy()

In [29]:
# Save the updated dataset with new features
dft3.to_csv("/content/clean_data_with_features.csv", index=False)

In [48]:
dft3.tail(50
    )


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,...,year_activ,month_activ,day_activ,days_since_activation,year_renewal,month_renewal,time_since_last_modif,contract_duration,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
14556,f6801c86377d1245947b37117551e1d8,foosdfpfkusacimwkcsosbicdxkicaua,69340,0,10929,3206.61,5195,0.0,62.26,0.144631,...,2009,11,27,5572,2015,11,2193,2557,-0.003038,0.177779
14557,5ec082b0ad9ce4bd1da77c7338fbd203,foosdfpfkusacimwkcsosbicdxkicaua,14611,3792,2839,1817.55,2839,0.0,37.45,0.164637,...,2011,7,18,4974,2015,7,1462,1827,-0.001547,0.177779
14558,5ab426a76ed5b8d67ba031a28f2ef195,lmkebamcaaclubfxadlmueccxoimlema,6207104,41532,558120,1773.22,0,0.0,16.38,0.143795,...,2011,2,21,5121,2015,2,1463,1826,-0.004111,-0.000001
14559,2e95bdbd205ebaf7f89c2a674e1f5281,foosdfpfkusacimwkcsosbicdxkicaua,4134,0,0,619.41,0,0.0,0.00,0.145137,...,2013,6,7,4284,2015,6,46,1096,-0.004347,0.177779
14560,4b8902f3ff0b41cc125d510bad0edbb3,MISSING,8114,46754,0,1217.55,0,0.0,16.42,0.144902,...,2009,10,8,5622,2015,10,2194,2557,-0.004847,0.177779
14561,37f06f14bb662b84f4f464322dce72db,MISSING,15191,0,420,1130.92,420,0.0,132.14,0.102772,...,2009,1,23,5880,2015,2,2215,2579,-0.005360,0.236694
14562,dd4e5b2dd42cefaa19cfd74f55baf5d1,MISSING,2349,0,0,292.14,0,0.0,10.36,0.165793,...,2009,7,29,5693,2015,7,2193,2557,-0.002411,0.177779
14563,7e2f957be04a83fcc92f97ee38709d14,lmkebamcaaclubfxadlmueccxoimlema,6395,0,0,642.73,0,0.0,131.64,0.114604,...,2009,9,1,5659,2015,9,7,2557,-0.008381,0.162916
14564,f49226b26ac8ff0003e734e13371247f,foosdfpfkusacimwkcsosbicdxkicaua,90714,0,6493,289.56,222,0.0,131.54,0.116910,...,2013,6,21,4270,2015,6,-132,1096,-0.009528,0.162916
14565,4e5950218d97027ecf18378c71442544,foosdfpfkusacimwkcsosbicdxkicaua,26024,0,4336,2810.26,4336,0.0,103.95,0.115237,...,2010,12,17,5187,2015,12,396,2192,-0.005839,0.000004


In [32]:
dft3.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_price_off_peak',
       'var_6m_price_peak', 'var_6m_price_mid_peak',

In [ ]:
#Preparing to Buid model

In [53]:
X, y = dft3.drop(['churn','id','channel_sales','year_activ','month_activ','day_activ','year_renewal','month_renewal','has_gas','origin_up'] , axis=1), dft3['churn']

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
#for column in X_train.columns:
    #try:
        #X_train[column].astype(float)  # Attempt to convert to float
    #except ValueError:
        #print(f"Column '{column}' contains non-numerical values:")
        #print(X_train[column].unique())  # Print unique values for inspection

#Building The Model


In [56]:
#Standarlisze the value
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [58]:
#Runing the data in Random forrest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [59]:
rdf = RandomForestClassifier(n_estimators=100, random_state=42)
rdf.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [60]:
rdf.score(X_train_scaled, y_train)

0.999914412872304

In [61]:
rdf.score(X_test_scaled, y_test)

0.8986995208761123

Providing ID of customer That I have high possibility to churn






In [64]:

# ... (your existing code) ...

# Predict churn on the test set
y_pred = rdf.predict(X_test_scaled)

# Create a DataFrame with customer IDs and predictions
churn_predictions_df = pd.DataFrame({'customer_id': X_test.index, 'churn_prediction': y_pred})

# Filter for customers predicted to churn
churn_customers_df = churn_predictions_df[churn_predictions_df['churn_prediction'] == 1]

# Display the table
print(churn_customers_df)

      customer_id  churn_prediction
158          3752                 1
355          3996                 1
407          9615                 1
587          5937                 1
917          1579                 1
961          1418                 1
991         10814                 1
1252        14350                 1
1335          217                 1
1383        10631                 1
1536         4948                 1
1668        10309                 1
1833         4900                 1
2526        14602                 1
2570          230                 1
2614         7532                 1
2641        12670                 1
